In [ ]:
# Copyright 2020 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 顶点客户端库：为在线 A/B 测试的自定义训练图像分类模型

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/master/notebooks/community/gapic/custom/showcase_custom_image_classification_online_ab_testing.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在 Colab 中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/master/notebooks/community/gapic/custom/showcase_custom_image_classification_online_ab_testing.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在 GitHub 上查看
    </a>
  </td>
</table>
<br/><br/><br/>

## 概述

本教程演示了如何使用Python的Vertex客户端库，为A/B测试训练和部署自定义图像分类模型进行在线预测。

数据集

本教程使用的数据集是来自[TensorFlow Datasets](https://www.tensorflow.org/datasets/catalog/overview)的[CIFAR10数据集](https://www.tensorflow.org/datasets/catalog/cifar10)。您将使用的数据集版本已经集成到TensorFlow中。训练模型预测图像属于十个类别中的哪一类：飞机、汽车、鸟、猫、鹿、狗、青蛙、马、船、卡车。

### 目标

在本教程中，您将学习如何从Python脚本中在Docker容器中使用Vertex客户端库创建自定义模型的多个实例，然后部署用于在线预测的A/B测试。
您还可以选择使用gcloud命令行或Google Cloud控制台在线创建自定义模型。

执行的步骤包括：

- 创建一个Vertex自定义作业，用于训练模型。
- 训练两个实例（A和B）的TensorFlow模型。
- 检索和加载模型工件。
- 查看模型评估。
- 将每个模型实例上传为Vertex `Model`资源。
- 部署模型实例到相同的服务`Endpoint`资源。
- 进行预测。
- 查看两个模型实例的结果。
- 取消部署`Model`资源。

### 成本

本教程使用 Google Cloud（GCP）的收费组件：

* Vertex AI
* Cloud Storage

了解[Vertex AI
定价](https://cloud.google.com/vertex-ai/pricing)和[Cloud Storage
定价](https://cloud.google.com/storage/pricing)，并使用[Pricing
Calculator](https://cloud.google.com/products/calculator/)
根据您的预计使用量生成成本估算。

安装

安装最新版本的Vertex客户端库。

In [ ]:
import os
import sys

# Google Cloud Notebook
if os.path.exists("/opt/deeplearning/metadata/env_version"):
    USER_FLAG = "--user"
else:
    USER_FLAG = ""

! pip3 install -U google-cloud-aiplatform $USER_FLAG

安装最新版本的*google-cloud-storage*库。

In [ ]:
! pip3 install -U google-cloud-storage $USER_FLAG

### 重新启动核心

一旦安装了Vertex客户端库和Google *cloud-storage*，您需要重新启动笔记本核心，以便它可以找到这些软件包。

In [ ]:
if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## 开始之前

### GPU运行时

*如果您的选项允许，确保在GPU运行时中运行此笔记本。在Colab中，选择* **运行时 > 更改运行时类型 > GPU**

### 设置您的Google云项目

**无论您的笔记本环境如何，以下步骤都是必需的。**

1. [选择或创建一个Google云项目](https://console.cloud.google.com/cloud-resource-manager)。当您首次创建一个帐户时，您将获得$300的免费信用用于计算/存储成本。

2. [确保对项目启用了计费](https://cloud.google.com/billing/docs/how-to/modify-project)。

3. [启用Vertex API和Compute Engine API](https://console.cloud.google.com/flows/enableapi?apiid=ml.googleapis.com,compute_component)。

4. [Google Cloud SDK](https://cloud.google.com/sdk) 已经安装在Google Cloud笔记本中。

5. 在下面的单元格中输入您的项目ID。然后运行该单元格，以确保Cloud SDK对本笔记本中的所有命令使用正确的项目。

**注意**：Jupyter以`!`为前缀运行带有的行作为Shell命令，并将带有`$`前缀的Python变量插值到这些命令中。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = !gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

#### 地区

您还可以更改 `REGION` 变量，该变量在本笔记本的其余部分中使用。 以下是 Vertex 支持的地区。 我们建议您选择最靠近您的地区。

- 美洲：`us-central1`
- 欧洲：`europe-west4`
- 亚太：`asia-east1`

您可能无法使用多区域存储桶来进行 Vertex 训练。 并非所有地区都支持所有 Vertex 服务。有关每个地区的最新支持信息，请参阅 [Vertex 位置文档](https://cloud.google.com/vertex-ai/docs/general/locations)

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

时间戳

如果您在直播教程会话中，可能会使用共享的测试账户或项目。为了避免资源创建时用户之间的名称冲突，您可以为每个实例会话创建一个时间戳，并将其附加到在本教程中将要创建的资源的名称上。

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

### 验证您的Google Cloud账户

**如果您正在使用Google Cloud Notebook**，您的环境已经通过验证。跳过这一步。

**如果您正在使用Colab**，运行下方的单元格并按照提示进行操作，通过oAuth验证您的账户。

**否则**，请按照以下步骤操作：

在Cloud控制台中，前往[创建服务帐号密钥](https://console.cloud.google.com/apis/credentials/serviceaccountkey)页面。

**点击创建服务帐号**。

在**服务帐号名称**字段中，输入一个名称，然后点击**创建**。

在**授予此服务帐号对项目的访问权限**部分，点击Role下拉列表。在筛选框中输入"Vertex"，并选择**Vertex管理员**。在筛选框中输入"Storage对象管理员"，并选择**Storage对象管理员**。

点击创建。一个包含您密钥的JSON文件将下载到您的本地环境中。

在下方的单元格中将您的服务帐号密钥路径输入为GOOGLE_APPLICATION_CREDENTIALS变量，并运行该单元格。

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

# If on Google Cloud Notebook, then don't execute this code
if not os.path.exists("/opt/deeplearning/metadata/env_version"):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

### 创建一个云存储桶

**无论您使用哪种笔记本环境，都需要按照以下步骤进行操作。**

当您使用Vertex客户端库提交自定义训练作业时，您需要将包含有训练代码的Python包上传到一个云存储桶中。Vertex将会从这个包中运行代码。在本教程中，Vertex还会将训练作业产生的训练模型保存在同一个桶中。然后您可以基于这个输出创建一个`Endpoint`资源，以便提供在线预测服务。

请在下面设置您的云存储桶的名称。存储桶的名称必须在所有谷歌云项目中全局唯一，包括您组织之外的项目。

In [ ]:
BUCKET_NAME = "gs://[your-bucket-name]"  # @param {type:"string"}

In [ ]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "gs://[your-bucket-name]":
    BUCKET_NAME = "gs://" + PROJECT_ID + "aip-" + TIMESTAMP

只有在您的存储桶不存在时才能运行以下单元格以创建您的云存储存储桶。

In [ ]:
! gsutil mb -l $REGION $BUCKET_NAME

最后，通过检查其内容来验证对您的云存储桶的访问权限。

In [ ]:
! gsutil ls -al $BUCKET_NAME

设置变量

接下来，设置一些在教程中使用的变量。
导入库并定义常量

#### 导入 Vertex 客户端库

将 Vertex 客户端库导入到我们的Python环境中。

In [ ]:
import time

from google.cloud.aiplatform import gapic as aip
from google.protobuf import json_format
from google.protobuf.struct_pb2 import Value

#### 顶点常数

为顶点设置以下常数：

- `API_ENDPOINT`：顶点API服务的数据集、模型、作业、流水线和终端服务端点。
- `PARENT`：顶点位置的根路径，用于数据集、模型、作业、流水线和终端资源。

In [ ]:
# API service endpoint
API_ENDPOINT = "{}-aiplatform.googleapis.com".format(REGION)

# Vertex location root path for your dataset, model and endpoint resources
PARENT = "projects/" + PROJECT_ID + "/locations/" + REGION

硬件加速器

设置硬件加速器（例如 GPU）用于训练和预测。

设置变量`TRAIN_GPU/TRAIN_NGPU`和`DEPLOY_GPU/DEPLOY_NGPU`以使用支持 GPU 的容器映像，并设置分配给虚拟机实例的 GPU 数量。例如，要使用一个 GPU 容器映像，为每个虚拟机分配 4 个 Nvidia Telsa K80 GPU，您可以指定:

（aip.AcceleratorType.NVIDIA_TESLA_K80, 4）

对于 GPU，可用的加速器包括：
   - aip.AcceleratorType.NVIDIA_TESLA_K80
   - aip.AcceleratorType.NVIDIA_TESLA_P100
   - aip.AcceleratorType.NVIDIA_TESLA_P4
   - aip.AcceleratorType.NVIDIA_TESLA_T4
   - aip.AcceleratorType.NVIDIA_TESLA_V100

否则，指定`(None, None)`以使用一个在 CPU 上运行的容器映像。

*注意*：TF 2.3 之前的版本对 GPU 的支持在加载本教程中的自定义模型时会失败。这是一个已知问题，在 TF 2.3 中已解决 -- 这是由生成服务函数中的静态图操作引起的。如果在您自己的自定义模型上遇到此问题，请使用支持 GPU 的 TF 2.3 容器映像。

In [ ]:
if os.getenv("IS_TESTING_TRAIN_GPU"):
    TRAIN_GPU, TRAIN_NGPU = (
        aip.AcceleratorType.NVIDIA_TESLA_K80,
        int(os.getenv("IS_TESTING_TRAIN_GPU")),
    )
else:
    TRAIN_GPU, TRAIN_NGPU = (aip.AcceleratorType.NVIDIA_TESLA_K80, 1)

if os.getenv("IS_TESTING_DEPOLY_GPU"):
    DEPLOY_GPU, DEPLOY_NGPU = (
        aip.AcceleratorType.NVIDIA_TESLA_K80,
        int(os.getenv("IS_TESTING_DEPOLY_GPU")),
    )
else:
    DEPLOY_GPU, DEPLOY_NGPU = (None, None)

容器（Docker）镜像

接下来，我们将设置用于训练和预测的Docker容器镜像

- TensorFlow 1.15
   - `gcr.io/cloud-aiplatform/training/tf-cpu.1-15:latest`
   - `gcr.io/cloud-aiplatform/training/tf-gpu.1-15:latest`
- TensorFlow 2.1
   - `gcr.io/cloud-aiplatform/training/tf-cpu.2-1:latest`
   - `gcr.io/cloud-aiplatform/training/tf-gpu.2-1:latest`
- TensorFlow 2.2
   - `gcr.io/cloud-aiplatform/training/tf-cpu.2-2:latest`
   - `gcr.io/cloud-aiplatform/training/tf-gpu.2-2:latest`
- TensorFlow 2.3
   - `gcr.io/cloud-aiplatform/training/tf-cpu.2-3:latest`
   - `gcr.io/cloud-aiplatform/training/tf-gpu.2-3:latest`
- TensorFlow 2.4
   - `gcr.io/cloud-aiplatform/training/tf-cpu.2-4:latest`
   - `gcr.io/cloud-aiplatform/training/tf-gpu.2-4:latest`
- XGBoost
   - `gcr.io/cloud-aiplatform/training/xgboost-cpu.1-1`
- Scikit-learn
   - `gcr.io/cloud-aiplatform/training/scikit-learn-cpu.0-23:latest`
- Pytorch
   - `gcr.io/cloud-aiplatform/training/pytorch-cpu.1-4:latest`
   - `gcr.io/cloud-aiplatform/training/pytorch-cpu.1-5:latest`
   - `gcr.io/cloud-aiplatform/training/pytorch-cpu.1-6:latest`
   - `gcr.io/cloud-aiplatform/training/pytorch-cpu.1-7:latest`

有关最新列表，请参阅[用于训练的预构建容器](https://cloud.google.com/vertex-ai/docs/training/pre-built-containers)。

- TensorFlow 1.15
   - `gcr.io/cloud-aiplatform/prediction/tf-cpu.1-15:latest`
   - `gcr.io/cloud-aiplatform/prediction/tf-gpu.1-15:latest`
- TensorFlow 2.1
   - `gcr.io/cloud-aiplatform/prediction/tf2-cpu.2-1:latest`
   - `gcr.io/cloud-aiplatform/prediction/tf2-gpu.2-1:latest`
- TensorFlow 2.2
   - `gcr.io/cloud-aiplatform/prediction/tf2-cpu.2-2:latest`
   - `gcr.io/cloud-aiplatform/prediction/tf2-gpu.2-2:latest`
- TensorFlow 2.3
   - `gcr.io/cloud-aiplatform/prediction/tf2-cpu.2-3:latest`
   - `gcr.io/cloud-aiplatform/prediction/tf2-gpu.2-3:latest`
- XGBoost
   - `gcr.io/cloud-aiplatform/prediction/xgboost-cpu.1-2:latest`
   - `gcr.io/cloud-aiplatform/prediction/xgboost-cpu.1-1:latest`
   - `gcr.io/cloud-aiplatform/prediction/xgboost-cpu.0-90:latest`
   - `gcr.io/cloud-aiplatform/prediction/xgboost-cpu.0-82:latest`
- Scikit-learn
   - `gcr.io/cloud-aiplatform/prediction/sklearn-cpu.0-23:latest`
   - `gcr.io/cloud-aiplatform/prediction/sklearn-cpu.0-22:latest`
   - `gcr.io/cloud-aiplatform/prediction/sklearn-cpu.0-20:latest`

有关最新列表，请参阅[用于预测的预构建容器](https://cloud.google.com/vertex-ai/docs/predictions/pre-built-containers)。

In [ ]:
if os.getenv("IS_TESTING_TF"):
    TF = os.getenv("IS_TESTING_TF")
else:
    TF = "2-1"

if TF[0] == "2":
    if TRAIN_GPU:
        TRAIN_VERSION = "tf-gpu.{}".format(TF)
    else:
        TRAIN_VERSION = "tf-cpu.{}".format(TF)
    if DEPLOY_GPU:
        DEPLOY_VERSION = "tf2-gpu.{}".format(TF)
    else:
        DEPLOY_VERSION = "tf2-cpu.{}".format(TF)
else:
    if TRAIN_GPU:
        TRAIN_VERSION = "tf-gpu.{}".format(TF)
    else:
        TRAIN_VERSION = "tf-cpu.{}".format(TF)
    if DEPLOY_GPU:
        DEPLOY_VERSION = "tf-gpu.{}".format(TF)
    else:
        DEPLOY_VERSION = "tf-cpu.{}".format(TF)

TRAIN_IMAGE = "gcr.io/cloud-aiplatform/training/{}:latest".format(TRAIN_VERSION)
DEPLOY_IMAGE = "gcr.io/cloud-aiplatform/prediction/{}:latest".format(DEPLOY_VERSION)

print("Training:", TRAIN_IMAGE, TRAIN_GPU, TRAIN_NGPU)
print("Deployment:", DEPLOY_IMAGE, DEPLOY_GPU, DEPLOY_NGPU)

#### 机器类型

接下来，设置用于训练和预测的机器类型。

- 设置变量`TRAIN_COMPUTE`和`DEPLOY_COMPUTE`来配置用于训练和预测的虚拟机的计算资源。
 - `机器类型`
     - `n1-standard`：每个虚拟CPU 3.75GB内存。
     - `n1-highmem`：每个虚拟CPU 6.5GB内存
     - `n1-highcpu`：每个虚拟CPU 0.9GB内存
 - `虚拟CPU数`：\[2, 4, 8, 16, 32, 64, 96 \]

*注意：以下不支持用于训练：*

 - `standard`：2个虚拟CPU
 - `highcpu`：2、4和8个虚拟CPU

*注意：您也可以使用n2和e2机器类型进行训练和部署，但它们不支持GPU*。

In [ ]:
if os.getenv("IS_TESTING_TRAIN_MACHINE"):
    MACHINE_TYPE = os.getenv("IS_TESTING_TRAIN_MACHINE")
else:
    MACHINE_TYPE = "n1-standard"

VCPU = "4"
TRAIN_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Train machine type", TRAIN_COMPUTE)

if os.getenv("IS_TESTING_DEPLOY_MACHINE"):
    MACHINE_TYPE = os.getenv("IS_TESTING_DEPLOY_MACHINE")
else:
    MACHINE_TYPE = "n1-standard"

VCPU = "4"
DEPLOY_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Deploy machine type", DEPLOY_COMPUTE)

# 教程

现在你已经准备好开始创建自己的自定义模型并训练 CIFAR10 数据集。

## 设置客户端

Vertex客户端库采用客户端/服务器模型。在您的一侧（Python脚本）中，您将创建一个客户端，该客户端发送请求并从Vertex服务器接收响应。

在本教程中，您将为工作流程中的不同步骤使用不同的客户端。因此，请提前设置它们全部。

- 用于`Model`资源的模型服务。
- 用于部署的端点服务。
- 用于批处理作业和自定义训练的作业服务。
- 用于提供服务的预测服务。

In [ ]:
# client options same for all services
client_options = {"api_endpoint": API_ENDPOINT}


def create_job_client():
    client = aip.JobServiceClient(client_options=client_options)
    return client


def create_model_client():
    client = aip.ModelServiceClient(client_options=client_options)
    return client


def create_endpoint_client():
    client = aip.EndpointServiceClient(client_options=client_options)
    return client


def create_prediction_client():
    client = aip.PredictionServiceClient(client_options=client_options)
    return client


clients = {}
clients["job"] = create_job_client()
clients["model"] = create_model_client()
clients["endpoint"] = create_endpoint_client()
clients["prediction"] = create_prediction_client()

for client in clients.items():
    print(client)

## 训练模型

有两种方法可以使用容器镜像来训练一个定制模型：

- **使用Google Cloud预构建的容器**。如果使用预构建的容器，您还需要指定一个要安装到容器镜像中的Python包。这个Python包包含了用于训练定制模型的代码。

- **使用自己的定制容器镜像**。如果使用自己的容器，容器需要包含用于训练定制模型的代码。

## 准备您的自定义工作规格

现在您的客户已经准备好了，您的第一步是为您的定制培训工作创建一个工作规格。工作规格将包括以下内容：

- `worker_pool_spec`：用于培训的机器类型的规格以及使用多少台机器（单台或分布式）
- `python_package_spec`：要与预构建容器一起安装的Python包的规格。

### 准备您的机器规格

现在为您的定制训练作业定义机器规格。这告诉 Vertex 需要为训练提供哪种类型的机器实例。
  - `machine_type`：要提供的 GCP 实例类型 -- 例如，n1-standard-8。
  - `accelerator_type`：硬件加速器的类型（如果有的话）。在本教程中，如果您之前设置了变量 `TRAIN_GPU != None`，则正在使用 GPU；否则将使用 CPU。
  - `accelerator_count`：加速器的数量。

In [ ]:
if TRAIN_GPU:
    machine_spec = {
        "machine_type": TRAIN_COMPUTE,
        "accelerator_type": TRAIN_GPU,
        "accelerator_count": TRAIN_NGPU,
    }
else:
    machine_spec = {"machine_type": TRAIN_COMPUTE, "accelerator_count": 0}

###准备您的磁盘规格

（可选）现在为您的自定义训练作业定义磁盘规格。这将告诉Vertex在每个机器实例中为训练提供什么类型和大小的磁盘。

- `boot_disk_type`：SSD或标准。SSD更快，标准较便宜。默认为SSD。
- `boot_disk_size_gb`：磁盘的大小，单位为GB。

In [ ]:
DISK_TYPE = "pd-ssd"  # [ pd-ssd, pd-standard]
DISK_SIZE = 200  # GB

disk_spec = {"boot_disk_type": DISK_TYPE, "boot_disk_size_gb": DISK_SIZE}

### 检查培训包

#### 包布局

在开始培训之前，您将查看如何为自定义培训任务组装Python包。解压缩后，该包包含以下目录/文件布局。

- PKG-INFO
- README.md
- setup.cfg
- setup.py
- trainer
  - \_\_init\_\_.py
  - task.py

文件 `setup.cfg` 和 `setup.py` 是将包安装到Docker镜像的操作环境中的指令。

文件 `trainer/task.py` 是执行自定义训练任务的Python脚本。*注意*，当我们在工作池规范中引用它时，我们将目录斜杠替换为一个点 (`trainer.task`) 并删除文件后缀 (`.py`)。

#### 包装

在下面的单元格中，您将组装训练包。

In [ ]:
# Make folder for Python training script
! rm -rf custom
! mkdir custom

# Add package information
! touch custom/README.md

setup_cfg = "[egg_info]\n\ntag_build =\n\ntag_date = 0"
! echo "$setup_cfg" > custom/setup.cfg

setup_py = "import setuptools\n\nsetuptools.setup(\n\n    install_requires=[\n\n        'tensorflow_datasets==1.3.0',\n\n    ],\n\n    packages=setuptools.find_packages())"
! echo "$setup_py" > custom/setup.py

pkg_info = "Metadata-Version: 1.0\n\nName: CIFAR10 image classification\n\nVersion: 0.0.0\n\nSummary: Demostration training script\n\nHome-page: www.google.com\n\nAuthor: Google\n\nAuthor-email: aferlitsch@google.com\n\nLicense: Public\n\nDescription: Demo\n\nPlatform: Vertex"
! echo "$pkg_info" > custom/PKG-INFO

# Make the training subfolder
! mkdir custom/trainer
! touch custom/trainer/__init__.py

#### Task.py 内容

在下一个单元格中，您可以写入训练脚本 task.py 的内容。我们不会详细讨论，这只是让您浏览一下。总结如下：

- 从命令行获取保存模型产物的目录（`--model_dir`），如果没有指定，则从环境变量 `AIP_MODEL_DIR` 中获取。
- 从 TF Datasets (tfds) 加载 CIFAR10 数据集。
- 使用 TF.Keras 模型 API 构建模型。
- 编译模型（`compile()`）。
- 根据参数 `args.distribute` 设置训练分发策略。
- 根据参数 `args.epochs` 和 `args.steps` 训练模型（`fit()`）。
- 将训练好的模型保存（`save(args.model_dir)`）到指定的模型目录中。

In [ ]:
%%writefile custom/trainer/task.py
# Single, Mirror and Multi-Machine Distributed Training for CIFAR-10

import tensorflow_datasets as tfds
import tensorflow as tf
from tensorflow.python.client import device_lib
import argparse
import os
import sys
tfds.disable_progress_bar()

parser = argparse.ArgumentParser()
parser.add_argument('--model-dir', dest='model_dir',
                    default=os.getenv("AIP_MODEL_DIR"), type=str, help='Model dir.')
parser.add_argument('--lr', dest='lr',
                    default=0.01, type=float,
                    help='Learning rate.')
parser.add_argument('--epochs', dest='epochs',
                    default=10, type=int,
                    help='Number of epochs.')
parser.add_argument('--steps', dest='steps',
                    default=200, type=int,
                    help='Number of steps per epoch.')
parser.add_argument('--distribute', dest='distribute', type=str, default='single',
                    help='distributed training strategy')
args = parser.parse_args()

print('Python Version = {}'.format(sys.version))
print('TensorFlow Version = {}'.format(tf.__version__))
print('TF_CONFIG = {}'.format(os.environ.get('TF_CONFIG', 'Not found')))
print('DEVICES', device_lib.list_local_devices())

# Single Machine, single compute device
if args.distribute == 'single':
    if tf.test.is_gpu_available():
        strategy = tf.distribute.OneDeviceStrategy(device="/gpu:0")
    else:
        strategy = tf.distribute.OneDeviceStrategy(device="/cpu:0")
# Single Machine, multiple compute device
elif args.distribute == 'mirror':
    strategy = tf.distribute.MirroredStrategy()
# Multiple Machine, multiple compute device
elif args.distribute == 'multi':
    strategy = tf.distribute.experimental.MultiWorkerMirroredStrategy()

# Multi-worker configuration
print('num_replicas_in_sync = {}'.format(strategy.num_replicas_in_sync))

# Preparing dataset
BUFFER_SIZE = 10000
BATCH_SIZE = 64


def make_datasets_unbatched():

  # Scaling CIFAR10 data from (0, 255] to (0., 1.]
  def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255.0
    return image, label


  datasets, info = tfds.load(name='cifar10',
                            with_info=True,
                            as_supervised=True)
  return datasets['train'].map(scale).cache().shuffle(BUFFER_SIZE).repeat()


# Build the Keras model
def build_and_compile_cnn_model():
  model = tf.keras.Sequential([
      tf.keras.layers.Conv2D(32, 3, activation='relu', input_shape=(32, 32, 3)),
      tf.keras.layers.MaxPooling2D(),
      tf.keras.layers.Conv2D(32, 3, activation='relu'),
      tf.keras.layers.MaxPooling2D(),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(10, activation='softmax')
  ])
  model.compile(
      loss=tf.keras.losses.sparse_categorical_crossentropy,
      optimizer=tf.keras.optimizers.SGD(learning_rate=args.lr),
      metrics=['accuracy'])
  return model


# Train the model
NUM_WORKERS = strategy.num_replicas_in_sync
# Here the batch size scales up by number of workers since
# `tf.data.Dataset.batch` expects the global batch size.
GLOBAL_BATCH_SIZE = BATCH_SIZE * NUM_WORKERS
train_dataset = make_datasets_unbatched().batch(GLOBAL_BATCH_SIZE)

with strategy.scope():
  # Creation of dataset, and model building/compiling need to be within
  # `strategy.scope()`.
  model = build_and_compile_cnn_model()

model.fit(x=train_dataset, epochs=args.epochs, steps_per_epoch=args.steps)
model.save(args.model_dir)

请将培训脚本存储在您的云存储桶中

接下来，您将培训文件夹打包成一个压缩的 tar 文件，然后存储在您的云存储桶中。

In [ ]:
! rm -f custom.tar custom.tar.gz
! tar cvf custom.tar custom
! gzip custom.tar
! gsutil cp custom.tar.gz $BUCKET_NAME/trainer_cifar10.tar.gz

### 定义模型A的工作人员池规范

接下来，您需要为自定义训练作业定义工作人员池规范。工作人员池规范将包括以下内容：

- `replica_count`：要配置此机器类型的实例数。
- `machine_spec`：硬件规范。
- `disk_spec`：（可选）磁盘存储规范。

- `python_package`：要安装在VM实例上的Python训练包，以及要调用的Python模块，以及Python模块的命令行参数。

现在让我们更深入地了解Python包规范：

- `executor_image_spec`：这是为您的自定义训练作业配置的Docker镜像。

- `package_uris`：这是要安装在配置实例上的Python训练包的位置（URIs）列表。这些位置需要在Cloud Storage存储桶中。这些可以是单独的Python文件，也可以是整个软件包的zip（归档）。在后一种情况下，作业服务将解压缩（解档）内容到Docker镜像中。

- `python_module`：要调用以运行自定义训练作业的Python模块（脚本）。在此示例中，您将调用`trainer.task.py` -- 请注意不需要附加`.py`后缀。

- `args`：要传递给相应Python模块的命令行参数。在此示例中，您将设置：
  - `"--model-dir=" + MODEL_DIR`：要存储模型工件的Cloud Storage位置。告知训练脚本保存模型工件的方式有两种：
      - 直接：将Cloud Storage位置作为命令行参数传递给训练脚本（设置变量`DIRECT = True`），或
      - 间接：服务将Cloud Storage位置作为环境变量`AIP_MODEL_DIR`传递给训练脚本（设置变量`DIRECT = False`）。在这种情况下，您需要在作业规范中告知服务模型工件的位置。
  - `"--epochs=" + EPOCHS`：训练的时代数量。
  - `"--steps=" + STEPS`：每个时代的步数（批次）。
  - `"--distribute=" + TRAIN_STRATEGY"`：用于单设备或分布式训练的训练分发策略。
     - `"single"`：单个设备。
     - `"mirror"`：单个计算实例上的所有GPU设备。
     - `"multi"`：所有计算实例上的所有GPU设备。

In [ ]:
JOB_NAME = "custom_job_A" + TIMESTAMP
MODEL_DIR = "{}/{}".format(BUCKET_NAME, JOB_NAME)
MODEL_DIR_A = MODEL_DIR

if not TRAIN_NGPU or TRAIN_NGPU < 2:
    TRAIN_STRATEGY = "single"
else:
    TRAIN_STRATEGY = "mirror"

EPOCHS = 20
STEPS = 100

DIRECT = True
if DIRECT:
    CMDARGS = [
        "--model-dir=" + MODEL_DIR,
        "--epochs=" + str(EPOCHS),
        "--steps=" + str(STEPS),
        "--distribute=" + TRAIN_STRATEGY,
    ]
else:
    CMDARGS = [
        "--epochs=" + str(EPOCHS),
        "--steps=" + str(STEPS),
        "--distribute=" + TRAIN_STRATEGY,
    ]

worker_pool_spec = [
    {
        "replica_count": 1,
        "machine_spec": machine_spec,
        "disk_spec": disk_spec,
        "python_package_spec": {
            "executor_image_uri": TRAIN_IMAGE,
            "package_uris": [BUCKET_NAME + "/trainer_cifar10.tar.gz"],
            "python_module": "trainer.task",
            "args": CMDARGS,
        },
    }
]

### 组装工作规范

现在组装完整的自定义工作规范描述：

- `display_name`：您为此自定义工作指定的可读名称。
- `job_spec`：自定义工作的规范。
    - `worker_pool_specs`：机器虚拟机实例的规范。
    - `base_output_directory`：这告诉服务在哪里保存模型工件的云存储位置（当变量`DIRECT = False`时）。然后服务将将位置传递给训练脚本作为环境变量`AIP_MODEL_DIR`，路径将如下形式：

                <output_uri_prefix>/model

In [ ]:
if DIRECT:
    job_spec = {"worker_pool_specs": worker_pool_spec}
else:
    job_spec = {
        "worker_pool_specs": worker_pool_spec,
        "base_output_directory": {"output_uri_prefix": MODEL_DIR},
    }

custom_job = {"display_name": JOB_NAME, "job_spec": job_spec}

### 训练模型

现在开始在Vertex上训练您的自定义训练作业。使用这个辅助函数`create_custom_job`，该函数接受以下参数：

-`custom_job`：自定义作业的规范。

辅助函数调用作业客户端服务的`create_custom_job`方法，将以下参数传入：

-`parent`：指向`Dataset`、`Model`和`Endpoint`资源的Vertex位置路径。
-`custom_job`：自定义作业的规范。

您将在`response`对象中显示返回的几个字段，其中最感兴趣的两个字段是：

`response.name`：为此自定义训练作业分配的Vertex完全限定标识符。您将保存此标识符以在后续步骤中使用。

`response.state`：自定义训练作业的当前状态。

In [ ]:
def create_custom_job(custom_job):
    response = clients["job"].create_custom_job(parent=PARENT, custom_job=custom_job)
    print("name:", response.name)
    print("display_name:", response.display_name)
    print("state:", response.state)
    print("create_time:", response.create_time)
    print("update_time:", response.update_time)
    return response


response = create_custom_job(custom_job)

现在获取您创建的自定义作业的唯一标识符。

In [ ]:
# The full unique ID for the custom job
job_id = response.name
# The short numeric ID for the custom job
job_short_id = job_id.split("/")[-1]

print(job_id)

获取有关自定义作业的信息

接下来，使用这个辅助函数 `get_custom_job`，它接受以下参数：

- `name`：自定义作业的 Vertex 完全限定标识符。

辅助函数调用作业客户端服务的 `get_custom_job` 方法，传入以下参数：

- `name`：自定义作业的 Vertex 完全限定标识符。

如果你记得，在调用 `create_custom_job` 方法时，你在 `response.name` 字段中获得了自定义作业的 Vertex 完全限定标识符，并将标识符保存在变量 `job_id` 中。

In [ ]:
def get_custom_job(name, silent=False):
    response = clients["job"].get_custom_job(name=name)
    if silent:
        return response

    print("name:", response.name)
    print("display_name:", response.display_name)
    print("state:", response.state)
    print("create_time:", response.create_time)
    print("update_time:", response.update_time)
    return response


response = get_custom_job(job_id)

## 等待训练完成

训练上述模型可能会花费20分钟以上的时间。

一旦您的模型训练完成，您可以通过将`end_time`减去`start_time`来计算训练模型所需的实际时间。对于您的模型，我们需要知道保存模型的位置，在Python脚本将其保存在您的本地云存储桶中的`MODEL_DIR + '/saved_model.pb'`。

In [ ]:
while True:
    response = get_custom_job(job_id, True)
    if response.state != aip.JobState.JOB_STATE_SUCCEEDED:
        print("Training job has not completed:", response.state)
        model_path_to_deploy_A = None
        if response.state == aip.JobState.JOB_STATE_FAILED:
            break
    else:
        if not DIRECT:
            MODEL_DIR_A = MODEL_DIR_A + "/model"
        model_path_to_deploy_A = MODEL_DIR_A
        print("Training Time:", response.update_time - response.create_time)
        break
    time.sleep(60)

print("model_to_deploy:", model_path_to_deploy_A)

### 定义 Model B 的 worker pool 规格

接下来，您需要定义自定义训练任务的 worker pool 规格。worker pool 规格将包括以下内容：

- `replica_count`：要创建的该机器类型的实例数量。
- `machine_spec`：硬件规格。
- `disk_spec`：（可选）磁盘存储规格。

In [ ]:
JOB_NAME = "custom_job_B" + TIMESTAMP
MODEL_DIR = "{}/{}".format(BUCKET_NAME, JOB_NAME)
MODEL_DIR_B = MODEL_DIR

if not TRAIN_NGPU or TRAIN_NGPU < 2:
    TRAIN_STRATEGY = "single"
else:
    TRAIN_STRATEGY = "mirror"

EPOCHS = 20
STEPS = 100

DIRECT = True
if DIRECT:
    CMDARGS = [
        "--model-dir=" + MODEL_DIR,
        "--epochs=" + str(EPOCHS),
        "--steps=" + str(STEPS),
        "--distribute=" + TRAIN_STRATEGY,
    ]
else:
    CMDARGS = [
        "--epochs=" + str(EPOCHS),
        "--steps=" + str(STEPS),
        "--distribute=" + TRAIN_STRATEGY,
    ]

worker_pool_spec = [
    {
        "replica_count": 1,
        "machine_spec": machine_spec,
        "disk_spec": disk_spec,
        "python_package_spec": {
            "executor_image_uri": TRAIN_IMAGE,
            "package_uris": [BUCKET_NAME + "/trainer_cifar10.tar.gz"],
            "python_module": "trainer.task",
            "args": CMDARGS,
        },
    }
]

### 组装工作规格

现在组装完整的自定义工作规格描述：

- `display_name`：您为此自定义工作分配的可阅读名称。
- `job_spec`：自定义工作的规格。
    - `worker_pool_specs`：机器 VM 实例的规格。
    - `base_output_directory`：这告诉服务将模型工件保存在哪个云存储位置（变量 `DIRECT = False` 时）。然后，服务将将位置传递给训练脚本作为环境变量 `AIP_MODEL_DIR`，路径将为以下形式：

                <output_uri_prefix>/model

In [ ]:
if DIRECT:
    job_spec = {"worker_pool_specs": worker_pool_spec}
else:
    job_spec = {
        "worker_pool_specs": worker_pool_spec,
        "base_output_directory": {"output_uri_prefix": MODEL_DIR},
    }

custom_job = {"display_name": JOB_NAME, "job_spec": job_spec}

### 训练模型

现在开始在Vertex上训练您的自定义训练工作。使用这个辅助函数`create_custom_job`，它接受以下参数：

- `custom_job`: 自定义作业的规范。

辅助函数调用作业客户端服务的`create_custom_job`方法，并带有以下参数：

- `parent`: 指向`Dataset`、`Model`和`Endpoint`资源的Vertex位置路径。
- `custom_job`: 自定义作业的规范。

您将在`response`对象中显示返回的一些字段，其中最感兴趣的两个是：

`response.name`: 分配给此自定义训练作业的Vertex完全限定标识符。您保存这个标识符以便在后续步骤中使用。

`response.state`: 自定义训练作业的当前状态。

In [ ]:
def create_custom_job(custom_job):
    response = clients["job"].create_custom_job(parent=PARENT, custom_job=custom_job)
    print("name:", response.name)
    print("display_name:", response.display_name)
    print("state:", response.state)
    print("create_time:", response.create_time)
    print("update_time:", response.update_time)
    return response


response = create_custom_job(custom_job)

现在获取您创建的自定义作业的唯一标识符。

In [ ]:
# The full unique ID for the custom job
job_id = response.name
# The short numeric ID for the custom job
job_short_id = job_id.split("/")[-1]

print(job_id)

## 等待训练完成

训练上述模型可能需要至少20分钟时间。

一旦您的模型训练完成，您可以通过将`end_time`减去`start_time`来计算实际训练模型所需的时间。对于您的模型，我们需要知道保存模型的位置，即Python脚本保存在您的本地云存储桶中的`MODEL_DIR + '/saved_model.pb'`。

In [ ]:
while True:
    response = get_custom_job(job_id, True)
    if response.state != aip.JobState.JOB_STATE_SUCCEEDED:
        print("Training job has not completed:", response.state)
        model_path_to_deploy_B = None
        if response.state == aip.JobState.JOB_STATE_FAILED:
            break
    else:
        if not DIRECT:
            MODEL_DIR_B = MODEL_DIR_B + "/model"
        model_path_to_deploy_B = MODEL_DIR_A
        print("Training Time:", response.update_time - response.create_time)
        break
    time.sleep(60)

print("model_to_deploy:", model_path_to_deploy_B)

加载保存的模型

您的模型实例以TensorFlow SavedModel格式存储在Cloud Storage桶中。让我们继续从Cloud Storage桶中加载它们，然后您可以执行一些操作，比如评估模型和进行预测。

要加载模型，您可以使用TF.Keras的`model.load_model()`方法，将保存模型的Cloud Storage路径传递给它--由`MODEL_DIR_A`和`MODEL_DIR_B`指定。

In [ ]:
import tensorflow as tf

model_A = tf.keras.models.load_model(MODEL_DIR_A)
model_B = tf.keras.models.load_model(MODEL_DIR_B)

## 评估模型

现在找出模型有多好。

### 加载评估数据

您将从 `tf.keras.datasets` 加载 CIFAR10 测试（留存）数据，使用 `load_data()` 方法。这将返回一个包含两个元素的元组数据集。第一个元素是训练数据，第二个是测试数据。每个元素也是一个包含两个元素的元组: 图像数据和对应的标签。

您不需要训练数据，因此我们将其加载为 `(_,_)`。

在您运行数据进行评估之前，您需要对其进行预处理:

x_test:
1. 将像素数据进行归一化（重新缩放），方法是将每个像素除以 255。这将用 0 到 1 之间的 32 位浮点数替换每个单字节整数像素。

y_test:
2. 标签当前是标量形式（稀疏）。如果您回顾 `trainer/task.py` 脚本中的 `compile()` 步骤，您会发现它是针对稀疏标签进行编译的。因此我们不需要做任何其他处理。

In [ ]:
import numpy as np
from tensorflow.keras.datasets import cifar10

(_, _), (x_test, y_test) = cifar10.load_data()
x_test = (x_test / 255.0).astype(np.float32)

print(x_test.shape, y_test.shape)

将客户端实例添加到模型输出。

对于A/B测试，每个模型需要在预测之外输出两个项目：

- 模型实例，无论是A还是B。
- 标识客户端会话的标识符，预测请求源自其中。

模型标识符已经内置到`predict()`方法返回的预测结果中，您可以使用模型标识符来确定是哪个模型实例A还是B进行了预测。

那么，为什么需要知道客户端会话呢？在A/B测试中，您不是在比较模型的客观表现 - 您已经在模型评估和持续评估后进行过比较了。您正在比较的是业务目标，比如客户是否点击了显示广告，是否选择了推荐内容，是否有交易转化等等。因此，业务目标是在客户会话上进行衡量的，您必须将模型实例与客户会话关联起来。

### 为A/B测试添加客户端会话输出

在TF.Keras Functional API中，当我们使用Model()类构建模型时，我们传递两个参数，输入张量和输出层；我称之为将所有内容连接起来连接输入到输出：

```
my_model = Model(inputs, outputs)
```

我们将使用这种方法在预测时实现传递客户端会话标识与训练好的模型实例。指定多个输入和输出的语法如下：

```
my_model = Model( inputs, [outputs1, outputs2])
```

这假设应用服务器，在进行预测请求时，将客户端会话ID添加到预测请求中。当应用服务器接收到预测响应时，将记录模型实例和客户端会话ID。然后，一个分析程序将处理这些记录，以衡量哪个模型A还是B更好地优化了业务目标。

### 构建包装模型

让我们开始吧。我们可以用三行Keras代码完成这个任务！

1. 创建一个`Lambda()`层。在这一层，您将从模型中的softmax输出获取输入。然后，您将输出softmax输出以及代表客户会话的数值标识符。因为这是一个模型，对于标识符，您需要：

- 以数字形式输出。
- 使用`tf.constant()`将值输出为图操作常量。
- 赋予一个张量形状（非标量），但指定值为列表，然后转换为张量。

2. 创建一个围绕原始模型的包装模型，其中：

- 输入为原始模型的输入。
- 输出为Lambda层。

在部署模型时，您将使用包装版本而不是原始模型。

In [ ]:
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.layers import Lambda

softmax = model_A.outputs[0]
outputs = Lambda(lambda z: (z, tf.convert_to_tensor([tf.constant(0)])))(softmax)
wrapper_model_A = Model(model_A.inputs, outputs)

softmax = model_B.outputs[0]
outputs = Lambda(lambda z: (z, tf.convert_to_tensor([tf.constant(1)])))(softmax)
wrapper_model_B = Model(model_B.inputs, outputs)

本地预测

现在让我们使用您的封装A/B模型之一进行本地预测。您将传递三个实例（图像）进行预测，并返回：

- 每个实例请求的softmax预测。
- 模型A/B标识符。在本例中为0表示A。

In [ ]:
wrapper_model_A.predict(x_test[0:3])

### 图像数据的服务功能

为了将图像传递给预测服务，您需要将压缩（例如JPEG）图像字节编码为base 64 -- 这样可以确保在网络上传输二进制数据时内容不被修改。由于部署的模型期望输入数据为原始（未压缩）字节，您需要确保base 64编码的数据在传递给部署的模型之前被转换回原始字节。

为了解决这个问题，定义一个服务功能（`serving_fn`）并将其附加到模型作为预处理步骤。添加一个`@tf.function`装饰器，这样服务功能将与底层模型融合（而不是在 CPU 上游）。

当您发送预测或解释请求时，请求的内容将被base 64解码为一个Tensorflow字符串（`tf.string`），然后传递给服务功能（`serving_fn`）。服务功能预处理`tf.string`为原始（未压缩）的numpy字节（`preprocess_fn`），以符合模型的输入要求：
- `io.decode_jpeg`- 解压缩JPG图像，返回一个具有三个通道（RGB）的Tensorflow张量。
- `image.convert_image_dtype` - 将整数像素值更改为浮点32，并将像素数据重新调整到0到1之间。
- `image.resize` - 调整图像大小以匹配模型的输入形状。

在这一点上，数据可以传递给模型（`m_call`）。

In [ ]:
CONCRETE_INPUT = "numpy_inputs"


def _preprocess(bytes_input):
    decoded = tf.io.decode_jpeg(bytes_input, channels=3)
    decoded = tf.image.convert_image_dtype(decoded, tf.float32)
    resized = tf.image.resize(decoded, size=(32, 32))
    return resized


@tf.function(input_signature=[tf.TensorSpec([None], tf.string)])
def preprocess_fn(bytes_inputs):
    decoded_images = tf.map_fn(
        _preprocess, bytes_inputs, dtype=tf.float32, back_prop=False
    )
    return {
        CONCRETE_INPUT: decoded_images
    }  # User needs to make sure the key matches model's input


@tf.function(input_signature=[tf.TensorSpec([None], tf.string)])
def serving_fn(bytes_inputs):
    images = preprocess_fn(bytes_inputs)
    prob = m_call(**images)
    return prob


m_call = tf.function(wrapper_model_A.call).get_concrete_function(
    [tf.TensorSpec(shape=[None, 32, 32, 3], dtype=tf.float32, name=CONCRETE_INPUT)]
)

tf.saved_model.save(
    wrapper_model_A, model_path_to_deploy_A, signatures={"serving_default": serving_fn}
)

In [ ]:
CONCRETE_INPUT = "numpy_inputs"


def _preprocess(bytes_input):  # noqa: 811
    decoded = tf.io.decode_jpeg(bytes_input, channels=3)
    decoded = tf.image.convert_image_dtype(decoded, tf.float32)
    resized = tf.image.resize(decoded, size=(32, 32))
    return resized


@tf.function(input_signature=[tf.TensorSpec([None], tf.string)])
def preprocess_fn(bytes_inputs):  # noqa: 811
    decoded_images = tf.map_fn(
        _preprocess, bytes_inputs, dtype=tf.float32, back_prop=False
    )
    return {
        CONCRETE_INPUT: decoded_images
    }  # User needs to make sure the key matches model's input


@tf.function(input_signature=[tf.TensorSpec([None], tf.string)])
def serving_fn(bytes_inputs):
    images = preprocess_fn(bytes_inputs)
    prob = m_call(**images)
    return prob


m_call = tf.function(wrapper_model_B.call).get_concrete_function(
    [tf.TensorSpec(shape=[None, 32, 32, 3], dtype=tf.float32, name=CONCRETE_INPUT)]
)

tf.saved_model.save(
    wrapper_model_B, model_path_to_deploy_B, signatures={"serving_default": serving_fn}
)

In [ ]:
loaded_A = tf.saved_model.load(model_path_to_deploy_A)
loaded_B = tf.saved_model.load(model_path_to_deploy_B)

serving_input = list(
    loaded_A.signatures["serving_default"].structured_input_signature[1].keys()
)[0]
print("Serving function input:", serving_input)

### 上传模型

使用这个帮助函数`upload_model`来上传您的模型，存储在SavedModel格式中，到`Model`服务，该服务将为您的模型实例化一个Vertex `Model`资源实例。一旦您完成了这个步骤，您就可以像任何其他Vertex `Model`资源实例一样使用这个`Model`资源实例，比如部署到一个用于提供预测的`Endpoint`资源上。

这个帮助函数接受以下参数：

- `display_name`：`Endpoint`服务的可读名称。
- `image_uri`：用于模型部署的容器镜像。
- `model_uri`：我们的SavedModel构件在Cloud Storage中的路径。对于本教程，这是`trainer/task.py`保存模型构件的Cloud Storage位置，我们在变量`MODEL_DIR`中指定了路径。

这个帮助函数调用`Model`客户端服务的`upload_model`方法，该方法接受以下参数：

- `parent`：`Dataset`、`Model`和`Endpoint`资源的Vertex根路径。
- `model`：Vertex `Model`资源实例的规范。

现在让我们更深入地了解Vertex模型规范`model`。这是一个字典对象，包含以下字段：

- `display_name`：`Model`资源的可读名称。
- `metadata_schema_uri`：由于您的模型是在没有Vertex `Dataset`资源的情况下构建的，因此您将将此字段留空(`''`)。
- `artificat_uri`：保存模型的SavedModel格式的Cloud Storage路径。
- `container_spec`：这是要安装在`Endpoint`资源上的Docker容器的规范，从中`Model`资源将提供预测。使用先前设置的变量`DEPLOY_GPU != None`来使用GPU；否则只分配CPU。

将模型上传到Vertex Model资源会返回一个长时间运行的操作，因为这可能需要一些时间。您调用response.result()，这是一个同步调用，当Vertex Model资源准备就绪时将返回。

帮助函数返回相应Vertex Model实例的Vertex完全限定标识符为upload_model_response.model。您将保存此标识符以备后续步骤使用，赋值给变量model_to_deploy_id。

In [ ]:
IMAGE_URI = DEPLOY_IMAGE


def upload_model(display_name, image_uri, model_uri):
    model = {
        "display_name": display_name,
        "metadata_schema_uri": "",
        "artifact_uri": model_uri,
        "container_spec": {
            "image_uri": image_uri,
            "command": [],
            "args": [],
            "env": [{"name": "env_name", "value": "env_value"}],
            "ports": [{"container_port": 8080}],
            "predict_route": "",
            "health_route": "",
        },
    }
    response = clients["model"].upload_model(parent=PARENT, model=model)
    print("Long running operation:", response.operation.name)
    upload_model_response = response.result(timeout=180)
    print("upload_model_response")
    print(" model:", upload_model_response.model)
    return upload_model_response.model


model_to_deploy_id_A = upload_model(
    "cifar10-" + TIMESTAMP, IMAGE_URI, model_path_to_deploy_A
)
model_to_deploy_id_B = upload_model(
    "cifar10-" + TIMESTAMP, IMAGE_URI, model_path_to_deploy_B
)

部署“模型”资源

现在部署经过训练的Vertex自定义“模型”资源。 这需要两个步骤：

1. 为部署“模型”资源创建一个“Endpoint”资源。

2. 将“模型”资源部署到“Endpoint”资源上。

### 创建`Endpoint`资源

使用这个辅助函数`create_endpoint`来创建一个用于部署模型以提供预测的Endpoint，带有以下参数：

- `display_name`：`Endpoint`资源的易读名称。

辅助函数使用端点客户服务的`create_endpoint`方法，该方法接受以下参数：

- `display_name`：`Endpoint`资源的易读名称。

创建`Endpoint`资源会返回一个长时间运行的操作，因为可能需要一些时间来为提供服务的`Endpoint`资源进行配置。您调用`response.result()`，这是一个同步调用，当Endpoint资源准备就绪时会返回。辅助函数返回`Endpoint`资源的Vertex完全限定标识符：`response.name`。

In [ ]:
ENDPOINT_NAME = "cifar10_endpoint-" + TIMESTAMP


def create_endpoint(display_name):
    endpoint = {"display_name": display_name}
    response = clients["endpoint"].create_endpoint(parent=PARENT, endpoint=endpoint)
    print("Long running operation:", response.operation.name)

    result = response.result(timeout=300)
    print("result")
    print(" name:", result.name)
    print(" display_name:", result.display_name)
    print(" description:", result.description)
    print(" labels:", result.labels)
    print(" create_time:", result.create_time)
    print(" update_time:", result.update_time)
    return result


result = create_endpoint(ENDPOINT_NAME)

现在获取您创建的“Endpoint”资源的唯一标识符。

In [ ]:
# The full unique ID for the endpoint
endpoint_id = result.name
# The short numeric ID for the endpoint
endpoint_short_id = endpoint_id.split("/")[-1]

print(endpoint_id)

### 计算实例扩展

您可以选择如下几种方式来扩展计算实例以处理您的在线预测请求：

- 单个实例：在线预测请求在一个计算实例上处理。
  - 将计算实例的最小（`MIN_NODES`）和最大（`MAX_NODES`）数量设定为一个。

- 手动扩展：在线预测请求被分配到您手动指定的固定数量的计算实例上。
  - 将计算实例的最小（`MIN_NODES`）和最大（`MAX_NODES`）数量设定为相同数量的节点。当一个模型首次部署到实例上时，固定数量的计算实例会被提供并且在线预测请求会均匀分配到它们上。

- 自动扩展：在线预测请求被分配到一个可扩展数量的计算实例上。
  - 在首次部署模型时，将计算实例的最小（`MIN_NODES`）数量设定为需要提供的计算实例数量，以及在负载条件下需要去除的计算实例数量，并将最大（`MAX_NODES`）数量设定为需要提供的计算实例数量。

计算实例的最小数量对应字段`min_replica_count`，最大计算实例数量对应字段`max_replica_count`，在您的后续部署请求中。

In [ ]:
MIN_NODES = 1
MAX_NODES = 1

### 部署 `Model` 资源到 `Endpoint` 资源

请使用这个辅助函数 `deploy_model` 来将从训练管道中上传（部署）的 `Model` 资源部署到你为提供预测创建的 `Endpoint` 资源，使用以下参数：

- `model`：要上传（部署）的模型的 Vertex 完全限定模型标识符。
- `deploy_model_display_name`：部署模型的人类可读名称。
- `endpoint`：要将模型部署到的 Vertex 完全限定的端点标识符。

辅助函数调用 `Endpoint` 客户端服务的方法 `deploy_model`，该方法接受以下参数：

- `endpoint`：要将 `Model` 资源部署到的 Vertex 完全限定的 `Endpoint` 资源标识符。
- `deployed_model`：用于部署模型的要求规范。
- `traffic_split`：在端点上流量的百分比将发送到该模型，指定为一个或多个键/值对的字典。
   - 如果只有一个模型，则指定为 **{ "0": 100 }**，其中 "0" 指代此模型被上传，100 表示流量的百分比为 100%。
   - 如果在端点上有现有的模型，流量将被分割给这些模型，那么需要使用 `model_id` 指定为 **{ "0": percent, model_id: percent, ... }**，其中 `model_id` 是已部署到端点的现有模型的模型 ID。这些百分比必须加起来为 100。

现在让我们深入了解 `deployed_model` 参数。这个参数被指定为一个 Python 字典，其中包含最少需要的字段：

- `model`：要部署的 Vertex 完全限定的（上传）模型标识符。
- `display_name`：部署模型的人类可读名称。
- `disable_container_logging`：这会禁用容器事件的记录，例如执行失败（默认为启用容器日志）。容器日志通常在调试部署时启用，然后在生产环境中部署时禁用。
- `dedicated_resources`：指的是为服务预测请求缩放的计算实例（副本）数量。
  - `machine_spec`：要预配的计算实例。使用之前设置的变量 `DEPLOY_GPU != None` 来使用 GPU；否则只分配 CPU。
  - `min_replica_count`：最初要预配的计算实例数量，这是之前设置为变量 `MIN_NODES` 的值。
  - `max_replica_count`：要扩展到的计算实例的最大数量，这是之前设置为变量 `MAX_NODES` 的值。

#### 流量分配

现在让我们深入了解 `traffic_split` 参数。这个参数被指定为一个 Python 字典。这可能一开始会有点令人困惑。让我解释一下，您可以将您的模型的多个实例部署到一个端点，然后设置每个实例接收的流量的百分比。

为什么要这样做呢？也许您已经在生产环境部署了之前的版本 -- 让我们称其为 v1。您在 v2 上得到了更好的模型评估，但您并不确定它是否真的比较好，直到部署到生产中为止。因此，在流量分割的情况下，您可能希望将 v2 部署到与 v1 相同的端点，但只分配给它 10% 的流量。这样，您可以在不影响大多数用户的情况下监视其表现，直到最终做出决定。

#### 响应

该方法返回一个长时间运行的操作 `response`。我们将等待这个操作同步完成，通过调用 `response.result()`，这会阻塞直到模型部署完成。如果这是第一次将模型部署到端点，可能需要额外几分钟来完成资源的预配。

In [ ]:
DEPLOYED_NAME = "cifar10_deployed-" + TIMESTAMP


def deploy_model(
    model, deployed_model_display_name, endpoint, traffic_split={"0": 100}
):

    # Accelerators can be used only if the model specifies a GPU image.
    if DEPLOY_GPU:
        machine_spec = {
            "machine_type": DEPLOY_COMPUTE,
            "accelerator_type": DEPLOY_GPU,
            "accelerator_count": DEPLOY_NGPU,
        }
    else:
        machine_spec = {
            "machine_type": DEPLOY_COMPUTE,
            "accelerator_count": 0,
        }

    deployed_model = {
        "model": model,
        "display_name": deployed_model_display_name,
        "dedicated_resources": {
            "min_replica_count": MIN_NODES,
            "max_replica_count": MAX_NODES,
            "machine_spec": machine_spec,
        },
    }

    response = clients["endpoint"].deploy_model(
        endpoint=endpoint, deployed_model=deployed_model, traffic_split=traffic_split
    )

    print("Long running operation:", response.operation.name)
    result = response.result()
    print("result")
    deployed_model = result.deployed_model
    print(" deployed_model")
    print("  id:", deployed_model.id)
    print("  model:", deployed_model.model)
    print("  display_name:", deployed_model.display_name)
    print("  create_time:", deployed_model.create_time)

    return deployed_model.id


deployed_model_id_A = deploy_model(
    model_to_deploy_id_A, DEPLOYED_NAME + "-A", endpoint_id, {"0": 100}
)
deployed_model_id_B = deploy_model(
    model_to_deploy_id_B,
    DEPLOYED_NAME + "-B",
    endpoint_id,
    {"0": 50, deployed_model_id_A: 50},
)

进行在线预测请求

现在对您部署的模型进行在线预测。

获取测试项

您将使用数据集的测试（预留）部分中的一个示例作为测试项。

In [ ]:
test_image = x_test[0]
test_label = y_test[0]
print(test_image.shape)

### 准备请求内容
您将发送CIFAR10图像作为压缩的JPG图像，而不是原始的未压缩字节：

- `cv2.imwrite`：使用openCV将未压缩的图像写入磁盘作为压缩的JPEG图像。
 - 将图像数据从\[0,1)范围反归一化为[0,255)。
 - 将32位浮点值转换为8位无符号整数。
- `tf.io.read_file`：将压缩的JPG图像作为原始字节读入内存。
- `base64.b64encode`：将原始字节编码为base 64编码字符串。

In [ ]:
import base64

import cv2

cv2.imwrite("tmp.jpg", (test_image * 255).astype(np.uint8))

bytes = tf.io.read_file("tmp.jpg")
b64str = base64.b64encode(bytes.numpy()).decode("utf-8")

### 发送预测请求

好的，现在你有一个测试图片。使用这个辅助函数 `predict_image`，它接受以下参数：

- `image`: 作为 numpy 数组的测试图片数据。
- `endpoint`: Vertex 中用于部署 `Model` 资源的完全限定标识符的 `Endpoint` 资源。
- `parameters_dict`: 用于服务的额外参数。

这个函数调用预测客户端服务 `predict` 方法，参数如下：

- `endpoint`: Vertex 中用于部署 `Model` 资源的完全限定标识符的 `Endpoint` 资源。
- `instances`: 要预测的一系列实例（编码后的图片）。
- `parameters`: 用于服务的额外参数。

为了将图片数据传递给预测服务，在上一步中，你将字节编码为 base64 -- 这样在通过网络传输二进制数据时，内容就不会被篡改。你需要告诉服务二进制代码你的模型部署到了哪里，并且内容已经被编码为 base64，这样服务二进制代码就会在另一端对其进行解码。

预测请求的每个实例是以下形式的字典项：

                        {serving_input: {'b64': content}}

- `input_name`: 底层模型的输入层名称。
- `'b64'`: 一个指示内容为 base64 编码的键。
- `content`: 作为 base64 编码字符串的压缩 JPG 图像字节。

由于 `predict()` 服务可以接受多个图片（实例），你将把你的单张图片发送为一个图片列表。作为最后一步，你将实例列表打包成谷歌的 protobuf 格式 -- 这是我们传递给 `predict()` 服务的内容。

`response` 对象返回一个列表，列表中每个元素对应请求中的对应图片。你会在每个预测的输出中看到：

- `predictions`: 对每个类别的预测的置信度水平，介于 0 和 1 之间。
- `output_2`: 客户端会话 ID。

In [ ]:
def predict_image(image, endpoint, parameters_dict):
    # The format of each instance should conform to the deployed model's prediction input schema.
    instances_list = [{serving_input: {"b64": image}}]
    instances = [json_format.ParseDict(s, Value()) for s in instances_list]

    response = clients["prediction"].predict(
        endpoint=endpoint, instances=instances, parameters=parameters_dict
    )
    print("response")
    print(" deployed_model_id:", response.deployed_model_id)
    predictions = response.predictions
    print("predictions")
    for prediction in predictions:
        print(" prediction:", dict(prediction))


predict_image(b64str, endpoint_id, None)

In [ ]:
def undeploy_model(deployed_model_id, endpoint):
    response = clients["endpoint"].undeploy_model(
        endpoint=endpoint, deployed_model_id=deployed_model_id, traffic_split={}
    )
    print(response)


undeploy_model(deployed_model_id_A, endpoint_id)
undeploy_model(deployed_model_id_B, endpoint_id)

清理工作

要清理本项目中使用的所有GCP资源，您可以删除用于本教程的[GCP项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除本教程中创建的各个资源：

- 数据集
- 流水线
- 模型
- 端点
- 批处理作业
- 自定义作业
- 超参数调整作业
- Cloud Storage 存储桶

In [ ]:
delete_dataset = True
delete_pipeline = True
delete_model = True
delete_endpoint = True
delete_batchjob = True
delete_customjob = True
delete_hptjob = True
delete_bucket = True

# Delete the dataset using the Vertex fully qualified identifier for the dataset
try:
    if delete_dataset and "dataset_id" in globals():
        clients["dataset"].delete_dataset(name=dataset_id)
except Exception as e:
    print(e)

# Delete the training pipeline using the Vertex fully qualified identifier for the pipeline
try:
    if delete_pipeline and "pipeline_id" in globals():
        clients["pipeline"].delete_training_pipeline(name=pipeline_id)
except Exception as e:
    print(e)

# Delete the model using the Vertex fully qualified identifier for the model
try:
    if delete_model and "model_to_deploy_id" in globals():
        clients["model"].delete_model(name=model_to_deploy_id)
except Exception as e:
    print(e)

# Delete the endpoint using the Vertex fully qualified identifier for the endpoint
try:
    if delete_endpoint and "endpoint_id" in globals():
        clients["endpoint"].delete_endpoint(name=endpoint_id)
except Exception as e:
    print(e)

# Delete the batch job using the Vertex fully qualified identifier for the batch job
try:
    if delete_batchjob and "batch_job_id" in globals():
        clients["job"].delete_batch_prediction_job(name=batch_job_id)
except Exception as e:
    print(e)

# Delete the custom job using the Vertex fully qualified identifier for the custom job
try:
    if delete_customjob and "job_id" in globals():
        clients["job"].delete_custom_job(name=job_id)
except Exception as e:
    print(e)

# Delete the hyperparameter tuning job using the Vertex fully qualified identifier for the hyperparameter tuning job
try:
    if delete_hptjob and "hpt_job_id" in globals():
        clients["job"].delete_hyperparameter_tuning_job(name=hpt_job_id)
except Exception as e:
    print(e)

if delete_bucket and "BUCKET_NAME" in globals():
    ! gsutil rm -r $BUCKET_NAME